# VAE Training - Faces dataset

In [1]:
#GPU tuning
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## imports

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from glob import glob
import shutil
import numpy as np

from models.VAE_a import VariationalAutoencoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [4]:
# run params
section = 'vae'
run_id = '0001'
data_name = 'faces'
RUN_FOLDER = 'run/{}/'.format(section)
RUN_FOLDER += '_'.join([run_id, data_name])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'load' #

DATA_FOLDER_0 = 'data/CelebA/'
DATA_FOLDER = os.path.join(DATA_FOLDER_0, 'celeba_1k') 

print(RUN_FOLDER)
print(DATA_FOLDER)

run/vae/0001_faces
data/CelebA/celeba_1k


## data

In [43]:
INPUT_DIM = (128,128,3)
BATCH_SIZE = 50
EPOCHS = 20

filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)
NUM_IMAGES 

962

In [48]:
# Dataset check 
move_flag = False 

if NUM_IMAGES % BATCH_SIZE !=0:
    print("WARNING: Dataset size not match with EPOCHS! extra:",  NUM_IMAGES % BATCH_SIZE )
    move_flag = True
    move_files = filenames[-(NUM_IMAGES % BATCH_SIZE):]       
else:
    print("Dataset OK")
    move_flag = False 

# Move extra files from Dataset
    
if move_flag:
    move_dir = os.path.join(DATA_FOLDER_0, 'move_dir')
    print("Move {:d} files to: {:s}".format(len(move_files), move_dir))
    ask = input("Proceed? (0 - NO)_")
    if ask != '0':
        if not os.path.exists(move_dir):
            os.mkdir(move_dir)
        for f in move_files:
            #print(f, os.path.join(move_dir, f.split('\\')[-1]))
            shutil.move(f, os.path.join(move_dir, f.split('\\')[-1]))
        print("move files - done")
    else:
        print("canceled")
    

Move 12 files to: data/CelebA/move_dir
Proceed? (0 - NO)1
move files - done


In [49]:
filenames = np.array(glob(os.path.join(DATA_FOLDER, '*/*.jpg')))

NUM_IMAGES = len(filenames)
NUM_IMAGES 

950

In [50]:
data_gen = ImageDataGenerator(rescale=1./255)

In [51]:
data_flow = data_gen.flow_from_directory(DATA_FOLDER
                                         , target_size = INPUT_DIM[:2]
                                         , batch_size = BATCH_SIZE
                                         , shuffle = True
                                         , class_mode = 'input'
                                         , subset = "training"
                                            )

Found 950 images belonging to 1 classes.


## architecture

In [52]:
vae = VariationalAutoencoder(
                input_dim = INPUT_DIM
                , encoder_conv_filters=[32,64,64, 64]
                , encoder_conv_kernel_size=[3,3,3,3]
                , encoder_conv_strides=[2,2,2,2]
                , decoder_conv_t_filters=[64,64,32,3]
                , decoder_conv_t_kernel_size=[3,3,3,3]
                , decoder_conv_t_strides=[2,2,2,2]
                , z_dim=200
                , use_batch_norm=True
                , use_dropout=True
                , r_loss_factor = 10000
                )

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights_'))

W1215 23:49:08.228910  6728 util.py:150] Unresolved object in checkpoint: (root).optimizer
W1215 23:49:08.228910  6728 util.py:150] Unresolved object in checkpoint: (root).loss
W1215 23:49:08.228910  6728 util.py:158] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/guide/checkpoint#loading_mechanics for details.


In [53]:
vae.encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 64, 64, 32)   896         encoder_input[0][0]              
__________________________________________________________________________________________________
batch_normalization_21 (BatchNo (None, 64, 64, 32)   128         encoder_conv_0[0][0]             
__________________________________________________________________________________________________
leaky_re_lu_21 (LeakyReLU)      (None, 64, 64, 32)   0           batch_normalization_21[0][0]     
____________________________________________________________________________________________

In [54]:
vae.decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 200)]             0         
_________________________________________________________________
dense_3 (Dense)              (None, 4096)              823296    
_________________________________________________________________
reshape_3 (Reshape)          (None, 8, 8, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 16, 16, 64)        36928     
_________________________________________________________________
batch_normalization_25 (Batc (None, 16, 16, 64)        256       
_________________________________________________________________
leaky_re_lu_25 (LeakyReLU)   (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 16, 16, 64)        0   

## training

In [55]:
LEARNING_RATE = 0.0005
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0 
EPOCHS

20

In [56]:
steps_per_epoch = NUM_IMAGES / BATCH_SIZE
steps_per_epoch

19.0

In [57]:
vae.compile(LEARNING_RATE)

In [59]:
vae.train_with_generator(     
    data_flow
    , epochs = EPOCHS
    , steps_per_epoch = NUM_IMAGES / BATCH_SIZE
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , save_every_n_batches = 20
    , initial_epoch = INITIAL_EPOCH
)

Epoch 1/20
19/19 [==============================] - ETA: 0s - loss: 405.3929 - reconstruction_loss: 341.7052 - kl_loss: 63.6877
Epoch 00001: saving model to run/vae/0001_faces\weights\weights
19/19 [==============================] - 2s 122ms/step - loss: 406.3905 - reconstruction_loss: 342.6037 - kl_loss: 63.7867
Epoch 2/20
19/19 [==============================] - ETA: 0s - loss: 401.9257 - reconstruction_loss: 337.8846 - kl_loss: 64.0411
Epoch 00002: saving model to run/vae/0001_faces\weights\weights
19/19 [==============================] - 2s 118ms/step - loss: 401.6696 - reconstruction_loss: 337.6788 - kl_loss: 63.9908
Epoch 3/20
19/19 [==============================] - ETA: 0s - loss: 397.4433 - reconstruction_loss: 332.7776 - kl_loss: 64.6656
Epoch 00003: saving model to run/vae/0001_faces\weights\weights
19/19 [==============================] - 2s 121ms/step - loss: 398.1548 - reconstruction_loss: 333.5357 - kl_loss: 64.6191
Epoch 4/20
19/19 [==============================] - ETA

In [18]:
#copy weghts
suff = '.data-00000-of-00001'

shutil.copy(os.path.join(RUN_FOLDER, 'weights/weights'+ suff), os.path.join(RUN_FOLDER, 'weights/weights_' + suff))


'run/vae/0001_faces\\weights/weights_2.data-00000-of-00001'